In [1]:
#imports
import sqlite3
import pandas as pd
import numpy as np
import regex as re
import matplotlib.pyplot as plt
from get_total_movie_info import *

# General Exploration of Data

Tables to explore info on the movies

## Load tables from data sources

Loading and looking at the tables and databases overall. Here we are trying to get a general idea of what we are working 
with


In [2]:
movie_gross = pd.read_csv('../Data/bom.movie_gross.csv')
movie_gross.head()

#domestic gross hard to read due to large numbers
#make domestic_gross abreviated as 'mil'

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [3]:
# connect to sql db
conn= sqlite3.connect("../Data/im.db")

In [4]:
#movi_basics table
q = """
SELECT *
FROM movie_basics
"""
movie_basics_df=pd.read_sql(q, conn)
movie_basics_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [5]:
#movie_ratings table
q = """
SELECT *
FROM movie_ratings
"""
movie_ratings_df=pd.read_sql(q, conn)
movie_ratings_df.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [6]:
#movie_akas table
q = """
SELECT * FROM movie_akas
"""
movie_akas_df = pd.read_sql(q, conn)
movie_akas_df.head()

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0


In [7]:
movie_budgets = pd.read_csv('../Data/tn.movie_budgets.csv')
movie_budgets.head()
#best table
#can get gross amounts
#forgeign gross = worldwide-domestic

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


# Mapping Tables

### Combining tables without movie names 

Runtime to rating: movie_basics to movie_ratings

* only about half of ht movies have ratings. Brain storming ideas on how to solve potential missing ratings

In [8]:
# Joinings movie basics and movie ratings tables on IDs
df = movie_ratings_df.merge(movie_basics_df, how='inner', on='movie_id')
df

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres
0,tt10356526,8.3,31,Laiye Je Yaarian,Laiye Je Yaarian,2019,117.0,Romance
1,tt10384606,8.9,559,Borderless,Borderless,2019,87.0,Documentary
2,tt1042974,6.4,20,Just Inès,Just Inès,2010,90.0,Drama
3,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy"
4,tt1060240,6.5,21,Até Onde?,Até Onde?,2011,73.0,"Mystery,Thriller"
...,...,...,...,...,...,...,...,...
73851,tt9805820,8.1,25,Caisa,Caisa,2018,84.0,Documentary
73852,tt9844256,7.5,24,Code Geass: Lelouch of the Rebellion - Glorifi...,Code Geass: Lelouch of the Rebellion Episode III,2018,120.0,"Action,Animation,Sci-Fi"
73853,tt9851050,4.7,14,Sisters,Sisters,2019,NaN,"Action,Drama"
73854,tt9886934,7.0,5,The Projectionist,The Projectionist,2019,81.0,Documentary


Need to filter out foreign movies

#### Looking at the movies_aka movies table to filter out regions

In [9]:
# Joinings movie basics, movie ratings, and movie_akas tables on IDs
df = movie_ratings_df.merge(movie_basics_df, how='inner', on='movie_id')
df= df.merge(movie_akas_df, how='inner', on = "movie_id")
df

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres,ordering,title,region,language,types,attributes,is_original_title
0,tt1042974,6.4,20,Just Inès,Just Inès,2010,90.0,Drama,1,Just Inès,None,None,original,None,1.0
1,tt1042974,6.4,20,Just Inès,Just Inès,2010,90.0,Drama,2,Samo Ines,RS,None,imdbDisplay,None,0.0
2,tt1042974,6.4,20,Just Inès,Just Inès,2010,90.0,Drama,3,Just Inès,GB,None,None,None,0.0
3,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",10,The Legend of Hercules,None,None,original,None,1.0
4,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",11,Hércules - A Lenda Começa,PT,None,imdbDisplay,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261801,tt9691896,6.3,21,Out,Coming Out,2018,64.0,Documentary,4,Coming Out,None,None,original,None,1.0
261802,tt9844256,7.5,24,Code Geass: Lelouch of the Rebellion - Glorifi...,Code Geass: Lelouch of the Rebellion Episode III,2018,120.0,"Action,Animation,Sci-Fi",1,Code Geass: Lelouch of the Rebellion Episode III,JP,None,None,None,0.0
261803,tt9844256,7.5,24,Code Geass: Lelouch of the Rebellion - Glorifi...,Code Geass: Lelouch of the Rebellion Episode III,2018,120.0,"Action,Animation,Sci-Fi",2,Code Geass: Lelouch of the Rebellion Episode III,None,None,original,None,1.0
261804,tt9844256,7.5,24,Code Geass: Lelouch of the Rebellion - Glorifi...,Code Geass: Lelouch of the Rebellion Episode III,2018,120.0,"Action,Animation,Sci-Fi",3,Code Geass: Lelouch of the Rebellion - Glorifi...,XWW,en,imdbDisplay,None,0.0


#### Keeping only the US movies

In [10]:
#filtering out foerign movies, showing all regions(countries)
df['region'].unique()

array([None, 'RS', 'GB', 'PT', 'JP', 'AL', 'HR', 'VN', 'IT', 'GR', 'UA',
       'RO', 'US', 'CA', 'CL', 'MX', 'RU', 'TR', 'IL', 'AR', 'PE', 'FR',
       'ES', 'BR', 'PL', 'BG', 'HU', 'VE', 'SE', 'XWW', 'XEU', 'SI', 'UY',
       'FI', 'DE', 'PH', 'NO', 'DK', 'EE', 'BE', 'LT', 'CO', 'CZ', 'AT',
       'GE', 'IE', 'ZA', 'BA', 'LV', 'AU', 'LU', 'IN', 'KR', 'IR', 'NL',
       'DZ', 'CN', 'NZ', 'ID', 'MY', 'JO', 'RW', 'EG', 'CH', 'LI', 'TH',
       'BF', 'CSXX', 'SG', 'NP', 'SK', 'BD', 'PA', 'KE', 'PK', 'PS', 'HK',
       'MK', 'KZ', 'TW', 'LK', 'IS', 'MW', 'DO', 'BO', 'PY', 'SL', 'AZ',
       'AF', 'AE', 'PR', 'CU', 'NG', 'IQ', 'AM', 'CY', 'GU', 'HN', 'XAS',
       'TN', 'BY', 'SA', 'HT', 'MT', 'BH', 'KH', 'KG', 'FO', 'SV', 'XSA',
       'GT', 'LB', 'BT', 'CR', 'XKV', 'UZ', 'MA', 'ME', 'LA', 'SB', 'XKO',
       'SY', 'TZ', 'QA', 'MN', 'EC', 'JM', 'NE', 'AN', 'MV', 'TT', 'ET',
       'CF', 'CM', 'MM', 'TJ', 'SM', 'MD', 'GW', 'KW', 'AD', 'BB', 'MZ',
       'BS', 'SO', 'BZ', 'IM', 'MC', 'AW', 

region we want to filter for is US

In [11]:
us_movies = df[df['region']=='US']
us_movies

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres,ordering,title,region,language,types,attributes,is_original_title
14,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",20,The Legend of Hercules,US,None,None,None,0.0
26,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",31,Untitled Hercules Project,US,None,working,None,0.0
29,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",5,Hercules: The Legend Begins,US,None,working,None,0.0
42,tt1094666,7.0,1613,The Hammer,Hamill,2010,108.0,"Biography,Drama,Sport",4,Hamill,US,None,festival,None,0.0
45,tt1094666,7.0,1613,The Hammer,Hamill,2010,108.0,"Biography,Drama,Sport",7,The Hammer,US,None,imdbDisplay,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261737,tt9165332,7.4,710,Chitralahari,Chitralahari,2019,131.0,Drama,2,Chitralahari,US,en,imdbDisplay,None,0.0
261760,tt9382596,4.1,293,The Car: Road to Revenge,The Car: Road to Revenge,2019,89.0,"Action,Horror,Thriller",1,The Car 2: Road to Revenge,US,None,working,None,0.0
261761,tt9382596,4.1,293,The Car: Road to Revenge,The Car: Road to Revenge,2019,89.0,"Action,Horror,Thriller",2,The Car 2,US,None,working,None,0.0
261762,tt9382596,4.1,293,The Car: Road to Revenge,The Car: Road to Revenge,2019,89.0,"Action,Horror,Thriller",3,The Car: Road to Revenge,US,None,None,None,0.0


Duplicate Movie IDs

In [12]:
#dropping duplicate movie_id's
us_movies= us_movies.drop_duplicates(subset=['movie_id'])
us_movies


,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres,ordering,title,region,language,types,attributes,is_original_title
14,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",20,The Legend of Hercules,US,None,None,None,0.0
42,tt1094666,7.0,1613,The Hammer,Hamill,2010,108.0,"Biography,Drama,Sport",4,Hamill,US,None,festival,None,0.0
64,tt1161457,4.2,148,Vanquisher,The Vanquisher,2016,90.0,"Action,Adventure,Sci-Fi",1,Vanquisher,US,None,None,new title,0.0
76,tt1171222,5.1,8296,Baggage Claim,Baggage Claim,2013,96.0,Comedy,5,Baggage Claim,US,None,None,None,0.0
85,tt1174693,5.8,2381,The Four-Faced Liar,The Four-Faced Liar,2010,87.0,"Comedy,Drama,Romance",5,The Four-Faced Liar,US,None,None,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261715,tt8948790,9.0,1778,Jersey,Jersey,2019,157.0,"Drama,Sport",3,Jersey,US,en,imdbDisplay,None,0.0
261734,tt9127488,9.3,10,Postal,Postal,2019,77.0,Comedy,2,Postal,US,None,None,None,0.0
261737,tt9165332,7.4,710,Chitralahari,Chitralahari,2019,131.0,Drama,2,Chitralahari,US,en,imdbDisplay,None,0.0
261760,tt9382596,4.1,293,The Car: Road to Revenge,The Car: Road to Revenge,2019,89.0,"Action,Horror,Thriller",1,The Car 2: Road to Revenge,US,None,working,None,0.0


No duplicate movies, shows regions as well

### Adding in gross numbers to the table

#### Clean title and year info to merge
-rules: 
1. all lower
2. only alphanumeric
3. remove spaces
4. use primary title as title

merging on title AND year to show differnce in the movies with the same name (remakes?)

In [13]:
# use RegEx to keep only alpha-numeric values, remove spaces and make all letters lowercase 
#us_movies["title"] = us_movies["title"].map(lambda x: re.sub(r'[^A-Za-z0-9 ]+', '', x).lower())
us_movies["title"] = us_movies["title"].map(lambda x: re.sub(r'[^A-Za-z0-9]+', '', x).lower())
movie_budgets["movie"] = movie_budgets["movie"].map(lambda x: re.sub(r'[^A-Za-z0-9]+', '', x).lower())
movie_gross["title"] = movie_gross["title"].map(lambda x: re.sub(r'[^A-Za-z0-9]+', '', x).lower())

<ipython-input-13-006c2fb176b4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_movies["title"] = us_movies["title"].map(lambda x: re.sub(r'[^A-Za-z0-9]+', '', x).lower())


In [14]:
#splitting the release_date column into only a year

In [15]:
# get a year column as int to match the other df types
# movie_budgets['year']= movie_budgets['release_date'].map(lambda x: int(x[-4:]))
# movie_budgets


#### rename columns for merging
##### dataframes merged to get gros income across

In [16]:
#movie_budgets.rename(columns={'movie': "title", 'domestic_gross':'domestic_gross_movie_budgets',
                             #'worldwide_gross': 'worldwide_gross_movie_budgets'}, inplace =True)
#movie_gross.rename(columns={'domestic_gross': 'domestic_gross_movie_gross', 'foreign_gross': 'foreign_movie_gross'}, inplace =True)

#us_movies.rename(columns ={'start_year': 'year'}, inplace=True)

movie_budgets.rename(columns={"movie": "title", "domestic_gross": "domestic_gross_movie_budgets",
                              "worldwide_gross": "worldwide_gross_movie_budgets"}, inplace=True)

movie_gross.rename(columns={"domestic_gross": "domestic_gross_movie_gross", 
                            "foreign_gross": "foreign_gross_movie_gross"}, inplace=True)

In [17]:
#combine tables with 2 columns to distinguish different movies with same name
final_gross_df1 = us_movies.merge(movie_budgets, on="title", how="inner")
final_gross_df1

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres,ordering,title,region,language,types,attributes,is_original_title,id,release_date,production_budget,domestic_gross_movie_budgets,worldwide_gross_movie_budgets
0,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",20,thelegendofhercules,US,None,None,None,0.0,42,"Jan 10, 2014","$70,000,000","$18,848,538","$58,953,319"
1,tt1171222,5.1,8296,Baggage Claim,Baggage Claim,2013,96.0,Comedy,5,baggageclaim,US,None,None,None,0.0,38,"Sep 27, 2013","$8,500,000","$21,569,509","$22,885,836"
2,tt1210166,7.6,326657,Moneyball,Moneyball,2011,133.0,"Biography,Drama,Sport",14,moneyball,US,None,imdbDisplay,None,0.0,15,"Sep 23, 2011","$50,000,000","$75,605,492","$111,300,835"
3,tt1212419,6.5,87288,Hereafter,Hereafter,2010,129.0,"Drama,Fantasy,Romance",4,hereafter,US,None,None,None,0.0,61,"Oct 15, 2010","$50,000,000","$32,746,941","$108,660,270"
4,tt6096334,6.4,5,Here After,Here After,2018,71.0,Documentary,1,hereafter,US,None,None,None,0.0,61,"Oct 15, 2010","$50,000,000","$32,746,941","$108,660,270"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684,tt2592614,5.6,74979,Resident Evil: The Final Chapter,Resident Evil: The Final Chapter,2016,107.0,"Action,Horror,Sci-Fi",2,residentevilthefinalchapter,US,None,None,None,0.0,84,"Jan 27, 2017","$40,000,000","$26,844,692","$314,101,190"
1685,tt2612800,5.7,37,Sisters,Schwestern,2013,84.0,Drama,2,sisters,US,None,imdbDisplay,None,0.0,57,"Dec 18, 2015","$30,000,000","$87,044,645","$106,030,660"
1686,tt2704998,7.0,163279,Game Night,Game Night,2018,100.0,"Action,Comedy,Crime",15,gamenight,US,None,imdbDisplay,None,0.0,94,"Feb 23, 2018","$37,000,000","$69,179,066","$117,378,084"
1687,tt3111864,5.0,102,8 Days,8 Days,2014,90.0,"Drama,Thriller",1,8days,US,None,None,None,0.0,48,"Oct 13, 2015","$2,500,000",$0,$0


In [18]:
final_gross_df2=us_movies.merge(movie_gross, on='title', how='inner').drop_duplicates(subset=['movie_id'])
final_gross_df2

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres,ordering,title,region,language,types,attributes,is_original_title,studio,domestic_gross_movie_gross,foreign_gross_movie_gross,year
0,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",20,thelegendofhercules,US,None,None,None,0.0,LG/S,18800000.0,42400000,2014
1,tt1171222,5.1,8296,Baggage Claim,Baggage Claim,2013,96.0,Comedy,5,baggageclaim,US,None,None,None,0.0,FoxS,21600000.0,887000,2013
2,tt1210166,7.6,326657,Moneyball,Moneyball,2011,133.0,"Biography,Drama,Sport",14,moneyball,US,None,imdbDisplay,None,0.0,Sony,75600000.0,34600000,2011
3,tt1212419,6.5,87288,Hereafter,Hereafter,2010,129.0,"Drama,Fantasy,Romance",4,hereafter,US,None,None,None,0.0,WB,32700000.0,72500000,2010
4,tt6096334,6.4,5,Here After,Here After,2018,71.0,Documentary,1,hereafter,US,None,None,None,0.0,WB,32700000.0,72500000,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,tt2592614,5.6,74979,Resident Evil: The Final Chapter,Resident Evil: The Final Chapter,2016,107.0,"Action,Horror,Sci-Fi",2,residentevilthefinalchapter,US,None,None,None,0.0,SGem,26800000.0,285400000,2017
1563,tt2612800,5.7,37,Sisters,Schwestern,2013,84.0,Drama,2,sisters,US,None,imdbDisplay,None,0.0,Uni.,87000000.0,18000000,2015
1564,tt2704998,7.0,163279,Game Night,Game Night,2018,100.0,"Action,Comedy,Crime",15,gamenight,US,None,imdbDisplay,None,0.0,WB (NL),69200000.0,48500000,2018
1565,tt7008872,7.0,18768,Boy Erased,Boy Erased,2018,115.0,"Biography,Drama",16,boyerased,US,None,imdbDisplay,None,0.0,Focus,6800000.0,5000000,2018


In [19]:
#shows all gross tables merged
final_gross_all = pd.concat([final_gross_df1, final_gross_df2]).drop_duplicates(subset=['movie_id'])
final_gross_all

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres,ordering,title,...,is_original_title,id,release_date,production_budget,domestic_gross_movie_budgets,worldwide_gross_movie_budgets,studio,domestic_gross_movie_gross,foreign_gross_movie_gross,year
0,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",20,thelegendofhercules,...,0.0,42.0,"Jan 10, 2014","$70,000,000","$18,848,538","$58,953,319",NaN,NaN,NaN,NaN
1,tt1171222,5.1,8296,Baggage Claim,Baggage Claim,2013,96.0,Comedy,5,baggageclaim,...,0.0,38.0,"Sep 27, 2013","$8,500,000","$21,569,509","$22,885,836",NaN,NaN,NaN,NaN
2,tt1210166,7.6,326657,Moneyball,Moneyball,2011,133.0,"Biography,Drama,Sport",14,moneyball,...,0.0,15.0,"Sep 23, 2011","$50,000,000","$75,605,492","$111,300,835",NaN,NaN,NaN,NaN
3,tt1212419,6.5,87288,Hereafter,Hereafter,2010,129.0,"Drama,Fantasy,Romance",4,hereafter,...,0.0,61.0,"Oct 15, 2010","$50,000,000","$32,746,941","$108,660,270",NaN,NaN,NaN,NaN
4,tt6096334,6.4,5,Here After,Here After,2018,71.0,Documentary,1,hereafter,...,0.0,61.0,"Oct 15, 2010","$50,000,000","$32,746,941","$108,660,270",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558,tt2132285,5.6,77726,The Bling Ring,The Bling Ring,2013,90.0,"Biography,Crime,Drama",1,theblingring,...,0.0,NaN,NaN,NaN,NaN,NaN,A24,5800000.0,13300000,2013.0
1559,tt2205697,7.2,78903,Stuck in Love.,Stuck in Love.,2012,97.0,"Comedy,Drama,Romance",6,stuckinlove,...,0.0,NaN,NaN,NaN,NaN,NaN,MNE,81100.0,NaN,2013.0
1561,tt2463288,6.0,48134,Walk of Shame,Walk of Shame,2014,95.0,Comedy,12,walkofshame,...,0.0,NaN,NaN,NaN,NaN,NaN,FCW,59200.0,NaN,2014.0
1565,tt7008872,7.0,18768,Boy Erased,Boy Erased,2018,115.0,"Biography,Drama",16,boyerased,...,0.0,NaN,NaN,NaN,NaN,NaN,Focus,6800000.0,5000000,2018.0


## Investigate Rotten Tomatoes Data

Joining director and release date

In [20]:
#putting director's name and movie name in one table
q = """
SELECT movie_basics.movie_id, movie_basics.primary_title, persons.primary_name
FROM directors
JOIN movie_basics on directors.movie_id = movie_basics.movie_id
JOIN persons ON directors.person_id = persons.person_id
"""
director_to_movies = pd.read_sql(q, conn).drop_duplicates(subset=['primary_name','primary_title'])
director_to_movies

,movie_id,primary_title,primary_name
0,tt0285252,Life's a Beach,Tony Vitale
1,tt0462036,Steve Phoenix: The Untold Story,Bill Haley
2,tt0835418,The Babymakers,Jay Chandrasekhar
4,tt0878654,Bulletface,Albert Pyun
5,tt0878654,Bulletface,Joe Baile
...,...,...,...
291166,tt8999974,Madre Luna,Daysi Burbano
291167,tt9001390,The woman and the river,Bernard Lessa
291168,tt9001494,Stone Mountain Georgia 08.28.18,Tate Nova
291169,tt9001494,Stone Mountain Georgia 08.28.18,Courtney Faye Powell


rt_move_info and rt_movie_reviews

In [21]:
#load rotten tomatoes
rt_movie_info = pd.read_table('../Data/rt.movie_info.tsv')
rt_movie_info.head()


,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


# *****need to unpack rows with multiple directors

In [22]:
for index, row in rt_movie_info.iterrows():
    print(row["director"])

William Friedkin
David Cronenberg
Allison Anders
Barry Levinson
Rodney Bennett
Jay Russell
Jake Kasdan
Ray Lawrence
Taylor Hackford
Frank Marshall
nan
nan
nan
William Friedkin
Peter Baldwin
George Hickenlooper
nan
Rick Rosenthal
Carl Erik Rinsch
Jim Jarmusch
nan
nan
Martyn Burke
Keith Gordon
John Woo
Michael Polish
Roy Ward Baker
David Arquette
John Sayles
Ken Loach
nan
William Wellman
Pauly Shore
Steve Boyum
Peter Cattaneo
Edward Dmytryk
Andy Sidaris
Ivan Nagy
Terence Young
Jon Turteltaub
nan
Paolo Sorrentino
Henning Schellerup
Allen Hughes|Albert Hughes
nan
Woody Allen
Yimou Zhang
Bill Froehlich
James Wong
nan
Pat Proft
Kevin Lima
Niels Arden Oplev
Matt Bettinelli-Olpin|Tyler Gillett
Anatole Litvak
Ira Miller
Steven Spielberg
Alan Alda
Otto Preminger
Terry Jones
Patrick Lussier
Tom Hanks
James Gray
Werner Herzog
Richard Kelly
Roy Rowland
nan
Ted Tetzlaff
William Beaudine
Bruce Beresford
nan
nan
Herbert Wilcox
John Gilling
Ernst Lubitsch
Cheryl Dunye
Ben Younger
Richard Linklater
Alan

Barnet Kellman
Andrew Davis
Stuart Heisler
Maclean Rogers
Ricki Stern|Anne Sundberg
Clint Eastwood
Jimmy Sangster
Stefan Ruzowitzky
Amir Motlagh
Michael Winterbottom
Ava DuVernay
Zhang Jun Zhao
Ridley Scott
Murray Fahey
Robert Ramirez
Tom DeCerchio|Tom De Cerchio
Cameron Crowe
Melvin Frank
Greg Beeman
Steven Hilliard Stern
Stephen Sommers
George Kaczender
Vikramaditya Motwane
Ciro Guerra
Norman Taurog
nan
Michael Curtiz
Joss Whedon
Gauri Shinde
nan
Burny Mattinson|Ron Clements|John Musker|Dave Michener
Sebastian Gutierrez
nan
Phil Alden Robinson
Les Rose
nan
Franco Brusati
Rick Rubin
Joel Coen
James Lapine
Barry Sonnenfeld
nan
Steve Barron
Gordon Douglas
David Mickey Evans
nan


In [23]:
x = rt_movie_info.dropna(subset=["director"])
x[x["director"].str.contains("\|")]

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
43,56,Albert Hughes and his brother Allen Hughes fol...,R,Drama,Allen Hughes|Albert Hughes,Michael Henry Brown|Albert Hughes|Allen Hughes,"Oct 6, 1995","May 19, 1998",NaN,NaN,119 minutes,NaN
53,67,"After a mysterious, lost night on their honeym...",R,Horror,Matt Bettinelli-Olpin|Tyler Gillett,Lindsay Devlin,"Jan 17, 2014","Apr 29, 2014",NaN,NaN,89 minutes,20th Century Fox
100,128,NaN,NR,Art House and International,Ye Ying|Zhang Li,NaN,NaN,NaN,NaN,NaN,120 minutes,NaN
121,155,"Taking place over the course of one night, The...",R,Comedy|Drama,Jennifer Jason Leigh|Alan Cumming,Jennifer Jason Leigh|Alan Cumming,"Jun 8, 2001","Jan 15, 2002",$,"2,974,308",115 minutes,Fine Line Features
146,191,Award-winning filmmaker Rakeysh Omprakash Mehr...,NR,Drama,Rakeysh Omprakash Mehra|Rakeysh Omprakesh Mehra,Prasoon Joshi,"Jul 12, 2013","Jan 12, 2015",$,"1,626,289",188 minutes,Reliance Entertainment
...,...,...,...,...,...,...,...,...,...,...,...,...
1490,1918,A collection of 15 chapters from Tarzan and th...,NR,Action and Adventure|Classics|Documentary,Henry MacRae|Harry McRae,William Lord Wright|Ian McCloskey Heath,"Oct 1, 1929","Oct 24, 2006",NaN,NaN,290 minutes,NaN
1511,1945,"Left on a nun's doorstep, Larry, Curly and Moe...",PG,Comedy,Bobby Farrelly|Peter Farrelly,Bobby Farrelly|Peter Farrelly|Mike Cerrone,"Apr 13, 2012","Jul 17, 2012",$,"41,800,000",92 minutes,20th Century Fox
1518,1953,A glimpse into the comedic process and private...,R,Comedy|Documentary|Television,Ricki Stern|Anne Sundberg,Ricki Stern,"Jun 11, 2010","Dec 14, 2010",$,"2,927,972",84 minutes,IFC Films
1529,1966,Two rabid Boston B-ball fans go over the edge ...,PG-13,Comedy,Tom DeCerchio|Tom De Cerchio,Judd Apatow|Colin Quinn,"Apr 19, 1996","Sep 3, 2002",NaN,NaN,90 minutes,NaN


In [24]:
# directors = rt_movie_info['director']
# rt_movie_info['director'] = directors.str.split(pat="|")

In [25]:
# testing unpacking the directors
direcs = []
ids = []

for index, row in rt_movie_info.iterrows():
    try:
        for person in row["director"]:
            direcs.append(person)
            ids.append(row["id"])
    except:
        direcs.append("-")
        ids.append(row["id"])
        
y = pd.DataFrame({"rt_id": ids, "director": direcs})
y[y["rt_id"].duplicated(keep=False)]

,rt_id,director
0,1,W
1,1,i
2,1,l
3,1,l
4,1,i
...,...,...
19516,1999,E
19517,1999,v
19518,1999,a
19519,1999,n


Splitting the theatre_date column into just year. Done to be able to merge rt_movie_info with total_movie_info

In [26]:
#drop missing values
rt_movie_info.dropna(subset=['theater_date'],inplace=True)
#split theatre_date column
rt_movie_info['year']= rt_movie_info['theater_date'].map(lambda x: int(x[-4:]))
rt_movie_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio,year
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN,1971
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One,2012
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN,1996
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN,1994
5,8,The year is 1942. As the Allies unite overseas...,PG,Drama|Kids and Family,Jay Russell,Gail Gilchriest,"Mar 3, 2000","Jul 11, 2000",NaN,NaN,95 minutes,Warner Bros. Pictures,2000


Plan: replace person ID with direct name and movie_id with the movie name. Then we will merge RT data with the gross table data. Merge on director and release date

## Merging the rotten tomatoes table with the director movie table

##### This will allow us to match the director with the release date


The goal in this is to match the director with the movie--- hopefully each director has only released one movie on one day

In [27]:
#merging direct_to_movies with final_gross_all
total_movie_info= final_gross_all.merge(director_to_movies[['movie_id','primary_name']], on="movie_id", how="left")
total_movie_info[total_movie_info["movie_id"].duplicated(keep=False)]
#checked to make sure duplicates all had different directors

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres,ordering,title,...,id,release_date,production_budget,domestic_gross_movie_budgets,worldwide_gross_movie_budgets,studio,domestic_gross_movie_gross,foreign_gross_movie_gross,year,primary_name
4,tt6096334,6.4,5,Here After,Here After,2018,71.0,Documentary,1,hereafter,...,61.0,"Oct 15, 2010","$50,000,000","$32,746,941","$108,660,270",NaN,NaN,NaN,NaN,Esy Casey
5,tt6096334,6.4,5,Here After,Here After,2018,71.0,Documentary,1,hereafter,...,61.0,"Oct 15, 2010","$50,000,000","$32,746,941","$108,660,270",NaN,NaN,NaN,NaN,Sarah Friedland
6,tt1232829,7.2,477771,21 Jump Street,21 Jump Street,2012,109.0,"Action,Comedy,Crime",26,21jumpstreet,...,44.0,"Mar 16, 2012","$42,000,000","$138,447,667","$202,812,429",NaN,NaN,NaN,NaN,Christopher Miller
7,tt1232829,7.2,477771,21 Jump Street,21 Jump Street,2012,109.0,"Action,Comedy,Crime",26,21jumpstreet,...,44.0,"Mar 16, 2012","$42,000,000","$138,447,667","$202,812,429",NaN,NaN,NaN,NaN,Phil Lord
12,tt1460743,6.9,6990,Reign of Assassins,Jian yu,2010,117.0,"Action,Adventure",11,reignofassassins,...,28.0,"Nov 1, 2016","$14,000,000",$0,"$11,354,230",NaN,NaN,NaN,NaN,John Woo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2552,tt2201884,8.0,5,The Workshop,The Workshop,2012,77.0,"Documentary,Family",1,theworkshop,...,NaN,NaN,NaN,NaN,NaN,Strand,22100.0,NaN,2018.0,Olivia Cuartero-Briggs
2562,tt8923482,7.5,103,One Child Nation,Born in China,2019,85.0,Documentary,3,borninchina,...,NaN,NaN,NaN,NaN,NaN,BV,13900000.0,11200000,2017.0,Nanfu Wang
2563,tt8923482,7.5,103,One Child Nation,Born in China,2019,85.0,Documentary,3,borninchina,...,NaN,NaN,NaN,NaN,NaN,BV,13900000.0,11200000,2017.0,Jialing Zhang
2571,tt5516328,6.4,23557,Ghost Stories,Ghost Stories,2017,98.0,"Drama,Horror",12,ghoststories,...,NaN,NaN,NaN,NaN,NaN,IFC,149000.0,NaN,2018.0,Jeremy Dyson


In [28]:
total_movie_info.head()

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres,ordering,title,...,id,release_date,production_budget,domestic_gross_movie_budgets,worldwide_gross_movie_budgets,studio,domestic_gross_movie_gross,foreign_gross_movie_gross,year,primary_name
0,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",20,thelegendofhercules,...,42.0,"Jan 10, 2014","$70,000,000","$18,848,538","$58,953,319",NaN,NaN,NaN,NaN,Renny Harlin
1,tt1171222,5.1,8296,Baggage Claim,Baggage Claim,2013,96.0,Comedy,5,baggageclaim,...,38.0,"Sep 27, 2013","$8,500,000","$21,569,509","$22,885,836",NaN,NaN,NaN,NaN,David E. Talbert
2,tt1210166,7.6,326657,Moneyball,Moneyball,2011,133.0,"Biography,Drama,Sport",14,moneyball,...,15.0,"Sep 23, 2011","$50,000,000","$75,605,492","$111,300,835",NaN,NaN,NaN,NaN,Bennett Miller
3,tt1212419,6.5,87288,Hereafter,Hereafter,2010,129.0,"Drama,Fantasy,Romance",4,hereafter,...,61.0,"Oct 15, 2010","$50,000,000","$32,746,941","$108,660,270",NaN,NaN,NaN,NaN,Clint Eastwood
4,tt6096334,6.4,5,Here After,Here After,2018,71.0,Documentary,1,hereafter,...,61.0,"Oct 15, 2010","$50,000,000","$32,746,941","$108,660,270",NaN,NaN,NaN,NaN,Esy Casey


In [29]:
total_movie_info[["release_date"]]

,release_date
0,"Jan 10, 2014"
1,"Sep 27, 2013"
2,"Sep 23, 2011"
3,"Oct 15, 2010"
4,"Oct 15, 2010"
...,...
2577,NaN
2578,NaN
2579,NaN
2580,NaN


Dropped duplicates but kept movies that appeared twice. Some movies will have 2 different directors.

We are going to merge based the assumption that no director/directors released more than 1 movie on the exact same day

### Final Merged Table

Gives us income, budget, genre, date, title, director, rating

In [30]:
#Dropping NAN values
total_movie_info.dropna(subset=['primary_name', 'release_date'], inplace= True)
rt_movie_info.dropna(subset=['director', 'theater_date'], inplace= True)

#### Cleaning before the merge

In [32]:
#making the primary_name uniform in im.db directors table
total_movie_info["primary_name"] = total_movie_info["primary_name"].map(lambda x: re.sub(r'[^A-Za-z0-9]+', '', x).lower())
#making the director names uniform in rt_movie_info table
rt_movie_info["director"] = rt_movie_info["director"].map(lambda x: re.sub(r'[^A-Za-z0-9]+', '', x).lower())

In [46]:
#merging total_movie_info to rt_movie_info
total_movie_info_all= total_movie_info.merge(rt_movie_info, left_on=['primary_name','release_date'],right_on=['director','theater_date'], how="left")
total_movie_info_all

,movie_id,averagerating,numvotes,primary_title,original_title,start_year,runtime_minutes,genres,ordering,title,...,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio_y,year_y
0,tt1043726,4.2,50352,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",20,thelegendofhercules,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt1171222,5.1,8296,Baggage Claim,Baggage Claim,2013,96.0,Comedy,5,baggageclaim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt1210166,7.6,326657,Moneyball,Moneyball,2011,133.0,"Biography,Drama,Sport",14,moneyball,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt1212419,6.5,87288,Hereafter,Hereafter,2010,129.0,"Drama,Fantasy,Romance",4,hereafter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt6096334,6.4,5,Here After,Here After,2018,71.0,Documentary,1,hereafter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1805,tt2612800,5.7,37,Sisters,Schwestern,2013,84.0,Drama,2,sisters,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1806,tt2704998,7.0,163279,Game Night,Game Night,2018,100.0,"Action,Comedy,Crime",15,gamenight,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1807,tt2704998,7.0,163279,Game Night,Game Night,2018,100.0,"Action,Comedy,Crime",15,gamenight,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1808,tt3111864,5.0,102,8 Days,8 Days,2014,90.0,"Drama,Thriller",1,8days,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
total_movie_info_all[["theater_date", "release_date", "director", "primary_name"]].head()

,theater_date,release_date,director,primary_name
0,NaN,"Jan 10, 2014",NaN,rennyharlin
1,NaN,"Sep 27, 2013",NaN,davidetalbert
2,NaN,"Sep 23, 2011",NaN,bennettmiller
3,NaN,"Oct 15, 2010",NaN,clinteastwood
4,NaN,"Oct 15, 2010",NaN,esycasey


In [50]:
def dollar_to_float(column):
    new_col = column.replace('[$,()]', '', regex=True).astype(float)
    
    return new_col

def handle_NaN(col1, col2):
    new_col = col1.fillna(col2)
#     new_col = new_col.fillna(0)
total_movie_info_all["clean_domestic_gross"] = handle_NaN(total_movie_info_all["domestic_gross_movie_budgets"], total_movie_info_all["domestic_gross_movie_gross"])
total_movie_info_all["clean_worldwide_gross"] = handle_NaN(total_movie_info_all["worldwide_gross_movie_budgets"], total_movie_info_all["foreign_gross_movie_gross"])
total_movie_info_all["clean_domestic_gross"] = dollar_to_float(total_movie_info_all["clean_domestic_gross"])
total_movie_info_all["clean_worldwide_gross"] = dollar_to_float(total_movie_info_all["clean_worldwide_gross"])
total_movie_info_all["production_budget"] = dollar_to_float(total_movie_info_all["production_budget"])

In [52]:
#checking for dupicate movie_id's
total_movie_info_all[["box_office", "domestic_gross_movie_budgets"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1810 entries, 0 to 1809
Data columns (total 2 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   box_office                    41 non-null     object
 1   domestic_gross_movie_budgets  1810 non-null   object
dtypes: object(2)
memory usage: 42.4+ KB


In [55]:
total_movie_info_all[["box_office", "domestic_gross_movie_budgets"]].dropna()

,box_office,domestic_gross_movie_budgets
37,"80,574,010","$80,574,010"
38,"53,021,560","$53,032,453"
75,"102,981,571","$103,068,524"
121,"303,001,229","$303,003,568"
146,"26,973,524","$27,007,844"
151,"320,676","$321,910"
163,"64,001,297","$64,003,625"
195,"21,379,315","$21,392,758"
202,"72,700,000","$75,624,550"
219,"98,000,000","$98,780,042"
